In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [64]:
x = 0.1
data = [[x]]
for _ in range(0, 10000):
    x = x + 1/10000
    data.append([x])
print(data[:5])

[[0.1], [0.10010000000000001], [0.10020000000000001], [0.10030000000000001], [0.10040000000000002]]


In [65]:
x_train = np.array(data[:int(len(data)*0.8)], dtype=np.float32)
y_train = np.array(data[1:int(len(data)*0.8)+1], dtype=np.float32)
x_test = np.array(data[int(len(data)*0.8)+1:len(data)-1], dtype=np.float32)
y_test = np.array(data[int(len(data)*0.8)+2:len(data)], dtype=np.float32)
print(len(x_test),len(y_test))

1999 1999


In [71]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__(
            i = L.Linear(1,1),
            h = L.Linear(1,1),
            o =  L.Linear(1,1)
        )
        
    def reset_state(self):
        self.h.reset_state()

    def __call__(self, x, y):
        i = self.i(x)
        h = self.h(i)
        o = self.o(h)
        self.loss = F.mean_squared_error(o,y)
        return self.loss

In [72]:
model = RNN()
optimizer = optimizers.AdaGrad()
optimizer.setup(model)

for step in range(201):
    x_variable = Variable(x_train)
    y_variable = Variable(y_train)
    optimizer.update(model, x_variable, y_variable)
    if step % 20 == 0:
        print(model.loss.data) 

0.16788852214813232
0.15109753608703613
0.14332501590251923
0.13749417662620544
0.13267797231674194
0.12851352989673615
0.12481343001127243
0.12146631628274918
0.11839918792247772
0.11556137353181839
0.11291591078042984


In [74]:
x_variable = Variable(x_test)
y_variable = Variable(y_test)
   
# model.reset_state()
model(x_variable,y_variable)
print(model.loss.data)

0.41856813430786133
